<a href="https://colab.research.google.com/github/Kim-se-yeon/o2o/blob/Practice/KcELECTRA_Base_Fine_Tune_checkpoint.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Package 설치 & 데이터 받기


## PyTorch-Lightning & Transformers🤗

PyTorch자체 외 다른 라이브러리들은 기존과 동일하게 사용할 수 있습니다.

# Inference

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive/')

In [ ]:
!nvidia-smi

Tue Aug  3 05:19:03 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.42.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P0    28W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!pip install tensorflow==2.2.0
!pip install PyYAML==5.4.1

try:
    import transformers, emoji, soynlp, pytorch_lightning
except:
    !pip install -U -q transformers emoji soynlp pytorch-lightning

     |████████████████████████████████| 516.2 MB 3.8 kB/s 
     |████████████████████████████████| 3.0 MB 44.1 MB/s 
     |████████████████████████████████| 2.9 MB 65.1 MB/s 
     |████████████████████████████████| 454 kB 67.4 MB/s 
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.5.0
    Uninstalling tensorflow-estimator-2.5.0:
      Successfully uninstalled tensorflow-estimator-2.5.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.5.0
    Uninstalling tensorboard-2.5.0:
      Successfully uninstalled tensorboard-2.5.0
  Attempting uninstall: h5py
    Found existing installation: h5py 3.1.0
    Uninstalling h5py-3.1.0:
      Successfully uninstalled h5py-3.1.0
  Attempting uninstall: gast
    Found existing installation: gast 0.4.0
    Uninstalling gast-0.4.0:
      Successfully uninstalled gast-0.4.0
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 2.5.0
    Uninstalli

In [ ]:
!git clone https://github.com/Kim-se-yeon/o2o

import zipfile
         
fantasy_zip = zipfile.ZipFile('/content/o2o/Total_dataset.zip')
fantasy_zip.extractall('/content/o2o')
 
fantasy_zip.close()

Cloning into 'o2o'...
remote: Enumerating objects: 74, done.
remote: Counting objects: 100% (74/74), done.
remote: Compressing objects: 100% (73/73), done.
remote: Total 74 (delta 28), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (74/74), done.


NSMC dataset + Navershopping dataset

In [ ]:
!head o2o/Total_train.txt

label	document
0	아 더빙.. 진짜 짜증나네요 목소리
1	흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나
0	너무재밓었다그래서보는것을추천한다
0	교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정
1	사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 던스트가 너무나도 이뻐보였다
0	막 걸음마 뗀 3세부터 초등학교 1학년생인 8살용영화.ㅋㅋㅋ...별반개도 아까움.
0	원작의 긴장감을 제대로 살려내지못했다.
0	별 반개도 아깝다 욕나온다 이응경 길용우 연기생활이몇년인지..정말 발로해도 그것보단 낫겟다 납치.감금만반복반복..이드라마는 가족도없다 연기못하는사람만모엿네
1	액션이 없는데도 재미 있는 몇안되는 영화


# 패키지 import & 기본 Args 설정

In [ ]:
import os
import pandas as pd

from pprint import pprint

import torch
from torch.utils.data import Dataset, DataLoader, TensorDataset
from torch.optim.lr_scheduler import ExponentialLR

from pytorch_lightning import LightningModule, Trainer, seed_everything

from transformers import AutoModelForSequenceClassification, AutoTokenizer, AdamW

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

import re
import emoji
from soynlp.normalizer import repeat_normalize

## 기본 학습 Arguments

In [ ]:
args = {
    'random_seed': 42, # Random Seed
    'pretrained_model': 'beomi/KcELECTRA-base',  # Transformers PLM name
    'pretrained_tokenizer': '',  # Optional, Transformers Tokenizer Name. Overrides `pretrained_model`
    'batch_size': 32,
    'lr': 5e-6,  # Starting Learning Rate
    'epochs': 16,  # Max Epochs
    'max_length': 150,  # Max Length input size
    'train_data_path': "o2o/Total_train.txt", #"nsmc/ratings_train.txt",  # Train Dataset file 
    'val_data_path': "o2o/Total_test.txt", #"nsmc/ratings_text.txt", # Validation Dataset file 
    'test_mode': False,  # Test Mode enables `fast_dev_run`
    'optimizer': 'AdamW',  # AdamW vs AdamP
    'lr_scheduler': 'exp',  # ExponentialLR vs CosineAnnealingWarmRestarts
    'fp16': True,  # Enable train on FP16(if GPU)
    'tpu_cores': 0,  # Enable TPU with 1 core or 8 cores
    'cpu_workers': os.cpu_count(),
}

In [ ]:
args

{'batch_size': 32,
 'cpu_workers': 4,
 'epochs': 16,
 'fp16': True,
 'lr': 5e-06,
 'lr_scheduler': 'exp',
 'max_length': 150,
 'optimizer': 'AdamW',
 'pretrained_model': 'beomi/KcELECTRA-base',
 'pretrained_tokenizer': '',
 'random_seed': 42,
 'test_mode': False,
 'tpu_cores': 0,
 'train_data_path': 'o2o/Total_train.txt',
 'val_data_path': 'o2o/Total_test.txt'}

# Model 만들기 with Pytorch Lightning

In [ ]:
class Model(LightningModule):
    def __init__(self, **kwargs):
        super().__init__()
        self.save_hyperparameters() # 이 부분에서 self.hparams에 위 kwargs가 저장된다.
        
        self.clsfier = AutoModelForSequenceClassification.from_pretrained(self.hparams.pretrained_model)
        self.tokenizer = AutoTokenizer.from_pretrained(
            self.hparams.pretrained_tokenizer
            if self.hparams.pretrained_tokenizer
            else self.hparams.pretrained_model
        )

    def forward(self, **kwargs):
        return self.clsfier(**kwargs)

    def step(self, batch, batch_idx):
        data, labels = batch
        output = self(input_ids=data, labels=labels)

        # Transformers 4.0.0+
        loss = output.loss
        logits = output.logits

        preds = logits.argmax(dim=-1)

        y_true = list(labels.cpu().numpy())
        y_pred = list(preds.cpu().numpy())

        return {
            'loss': loss,
            'y_true': y_true,
            'y_pred': y_pred,
        }

    def training_step(self, batch, batch_idx):
        return self.step(batch, batch_idx)

    def validation_step(self, batch, batch_idx):
        return self.step(batch, batch_idx)

    def epoch_end(self, outputs, state='train'):
        loss = torch.tensor(0, dtype=torch.float)
        for i in outputs:
            loss += i['loss'].cpu().detach()
        loss = loss / len(outputs)

        y_true = []
        y_pred = []
        for i in outputs:
            y_true += i['y_true']
            y_pred += i['y_pred']
        
        acc = accuracy_score(y_true, y_pred)
        prec = precision_score(y_true, y_pred)
        rec = recall_score(y_true, y_pred)
        f1 = f1_score(y_true, y_pred)

        self.log(state+'_loss', float(loss), on_epoch=True, prog_bar=True)
        self.log(state+'_acc', acc, on_epoch=True, prog_bar=True)
        self.log(state+'_precision', prec, on_epoch=True, prog_bar=True)
        self.log(state+'_recall', rec, on_epoch=True, prog_bar=True)
        self.log(state+'_f1', f1, on_epoch=True, prog_bar=True)
        print(f'[Epoch {self.trainer.current_epoch} {state.upper()}] Loss: {loss}, Acc: {acc}, Prec: {prec}, Rec: {rec}, F1: {f1}')
        return {'loss': loss}
    
    def training_epoch_end(self, outputs):
        self.epoch_end(outputs, state='train')

    def validation_epoch_end(self, outputs):
        self.epoch_end(outputs, state='val')

    def configure_optimizers(self):
        if self.hparams.optimizer == 'AdamW':
            optimizer = AdamW(self.parameters(), lr=self.hparams.lr)
        elif self.hparams.optimizer == 'AdamP':
            from adamp import AdamP
            optimizer = AdamP(self.parameters(), lr=self.hparams.lr)
        else:
            raise NotImplementedError('Only AdamW and AdamP is Supported!')
        if self.hparams.lr_scheduler == 'cos':
            scheduler = CosineAnnealingWarmRestarts(optimizer, T_0=1, T_mult=2)
        elif self.hparams.lr_scheduler == 'exp':
            scheduler = ExponentialLR(optimizer, gamma=0.5)
        else:
            raise NotImplementedError('Only cos and exp lr scheduler is Supported!')
        return {
            'optimizer': optimizer,
            'scheduler': scheduler,
        }

    def read_data(self, path):
        if path.endswith('xlsx'):
            return pd.read_excel(path)
        elif path.endswith('csv'):
            return pd.read_csv(path)
        elif path.endswith('tsv') or path.endswith('txt'):
            return pd.read_csv(path, sep='\t')
        else:
            raise NotImplementedError('Only Excel(xlsx)/Csv/Tsv(txt) are Supported')

    def clean(self, x):
        emojis = ''.join(emoji.UNICODE_EMOJI.keys())
        pattern = re.compile(f'[^ .,?!/@$%~％·∼()\x00-\x7Fㄱ-힣{emojis}]+')
        url_pattern = re.compile(
            r'https?:\/\/(www\.)?[-a-zA-Z0-9@:%._\+~#=]{1,256}\.[a-zA-Z0-9()]{1,6}\b([-a-zA-Z0-9()@:%_\+.~#?&//=]*)')
        x = pattern.sub(' ', x)
        x = url_pattern.sub('', x)
        x = x.strip()
        x = repeat_normalize(x, num_repeats=2)
        return x

    def encode(self, x, **kwargs):
        return self.tokenizer.encode(
            self.clean(str(x)),
            padding='max_length',
            max_length=self.hparams.max_length,
            truncation=True,
            **kwargs,
        )

    def preprocess_dataframe(self, df):
        df['document'] = df['document'].map(self.encode)
        return df

    def dataloader(self, path, shuffle=False):
        df = self.read_data(path)
        df = self.preprocess_dataframe(df)

        dataset = TensorDataset(
            torch.tensor(df['document'].to_list(), dtype=torch.long),
            torch.tensor(df['label'].to_list(), dtype=torch.long),
        )
        return DataLoader(
            dataset,
            batch_size=self.hparams.batch_size * 1 if not self.hparams.tpu_cores else self.hparams.tpu_cores,
            shuffle=shuffle,
            num_workers=self.hparams.cpu_workers,
        )

    def train_dataloader(self):
        return self.dataloader(self.hparams.train_data_path, shuffle=True)

    def val_dataloader(self):
        return self.dataloader(self.hparams.val_data_path, shuffle=False)


In [ ]:
from pytorch_lightning.callbacks import ModelCheckpoint

checkpoint_callback = ModelCheckpoint(
    filename='epoch{epoch}-val_acc{val_acc:.4f}',
    monitor='val_acc',
    save_top_k=3,
    mode='max',
    auto_insert_metric_name=False,
)

# 학습!

> 💡**NOTE**💡 1epoch별로 GPU P100기준 약50분, GPU V100기준 ~15분이 걸립니다. <br>
> 학습시 약 0.92 이하의 validation acc를 얻을 수 있습니다.

> Update @ 2020.09.01
> 최근 Colab Pro에서 V100이 배정됩니다.

```python
# 1epoch
loss=0.207, v_num=0, val_loss=0.221, val_acc=0.913, val_precision=0.914, val_recall=0.913, val_f1=0.914
# 2epoch
loss=0.152, v_num=0, val_loss=0.213, val_acc=0.918, val_precision=0.912, val_recall=0.926, val_f1=0.919
# 3epoch
loss=0.135, v_num=0, val_loss=0.225, val_acc=0.919, val_precision=0.907, val_recall=0.936, val_f1=0.921
```

# Inference

In [ ]:
checkpoint = torch.load('/content/gdrive/My Drive/KcELECTRA_16epoch_3try_state_dict.pt')

FileNotFoundError: ignored

In [ ]:
model = Model(**args)
model.load_state_dict(torch.load('/content/gdrive/My Drive/KcELECTRA_16epoch_3try_state_dict.pt'))
model.eval()

In [ ]:
def infer(x):
  result = torch.softmax(model(**model.tokenizer(x, return_tensors='pt')).logits, dim=-1)
  probability = float(result[0][0])
  if(probability > 0.5):
    print("{:.2f}% 확률로 긍정 리뷰입니다.".format(probability * 100))
  else:
    print("{:.2f}% 확률로 부정 리뷰입니다.".format((1-probability) * 100))
  return 
   

In [ ]:
infer('보통이에요')

In [ ]:
infer('나쁘지 않아요')

In [ ]:
infer('시간맞춰와쥬셔서 감사합니다^^^^^^^^')
#★★★

In [ ]:
infer('시간맞춰와쥬셔서 감사합니다')
#★★★

In [ ]:
infer('시간맞춰와주셔서 감사합니다^^^^^^^^')
#★★★

In [ ]:
infer('시간맞춰와주셔서 감사합니다')
#★★★

In [ ]:
infer('약속 시간도 잘 지켜주시고 아기도 예뻐해주시고 감사했어요😊 앞으로 자주 뵙고 싶어요 💓')
#★★★★★

In [ ]:
infer('약속 시간도 잘 지켜주시고 아기도 예뻐해주시고 감사했어요 앞으로 자주 뵙고 싶어요 ')
#★★★★★

In [ ]:
infer('좋은지 모르겠어요')

In [ ]:
infer('정말 이런 경험 오랜만이네요')

In [ ]:
infer('뭐라 말해야 될지 모르겠네요')

In [ ]:
infer('그럭저럭이에요')

In [ ]:
infer('애매한거 같아요')

In [ ]:
infer('사람다웠어여요')

In [ ]:
infer('테이블에서 음식을 먹고나면 깨끗하게 치웠으면 좋겠네요')

In [ ]:
infer('테이블에서 음식을 먹고나면 깨끗하게 치웠으면 좋겠네?')

In [ ]:
infer('')